## Compile the redox potentials
We want to see what the energy differences for different methods are


In [1]:
import pandas as pd
from ase.db import connect

## Load the Database into Pandas
Get the energies for each system for each NWChem setting

In [2]:
data = []
with connect('data.db') as db:
    for row in db.select(''):
        record = row.key_value_pairs.copy()
        record['energy'] = row.energy
        data.append(record)
data = pd.DataFrame(data)
print(f'Loaded {len(data)} computations')

Loaded 31 computations


## Compute the redox potentials
Get the energy differences between different stats

In [3]:
method_keys = ['basis_set', 'xc']
records = []
for level, subset in data.groupby(method_keys):
    method_desc = dict(zip(method_keys, level))
    
    # Match states
    neutrals = subset.query('state=="neutral"')
    for state in ['oxidized', 'reduced']:
        redoxed = subset.query(f'state=="{state}"')
        merged = redoxed.merge(neutrals, on='inchi_key')
        
        # Create a record for each
        for _, row in merged.iterrows():
            record = {}
            record['inchi_key'] = row['inchi_key']
            record['state'] = state
            record['potential'] = row['energy_x'] - row['energy_y']
            record['runtime'] = row['runtime_x'] + row['runtime_y']
            record.update(method_desc)
            records.append(record)
records = pd.DataFrame(records)

In [4]:
records

,inchi_key,state,potential,runtime,basis_set,xc
0,VNWKTOKETHGBQD-UHFFFAOYSA-N,oxidized,12.661882,29.321826,aug-cc-pvdz,B3LYP
1,IJDNQMDRQITEOD-UHFFFAOYSA-N,oxidized,10.662913,321.096845,aug-cc-pvdz,B3LYP
2,QLNJFJADRCOGBJ-UHFFFAOYSA-N,oxidized,9.430034,628.736598,aug-cc-pvdz,B3LYP
3,FPOLWERNILTNDK-UHFFFAOYSA-N,oxidized,9.160787,464.398307,aug-cc-pvdz,B3LYP
4,VNWKTOKETHGBQD-UHFFFAOYSA-N,reduced,0.646431,11.909188,aug-cc-pvdz,B3LYP
5,IJDNQMDRQITEOD-UHFFFAOYSA-N,reduced,0.567407,203.174535,aug-cc-pvdz,B3LYP
6,QLNJFJADRCOGBJ-UHFFFAOYSA-N,reduced,0.284250,541.772185,aug-cc-pvdz,B3LYP
7,VNWKTOKETHGBQD-UHFFFAOYSA-N,oxidized,12.634582,18.203282,cc-pvdz,B3LYP
8,IJDNQMDRQITEOD-UHFFFAOYSA-N,oxidized,10.597766,101.392672,cc-pvdz,B3LYP
9,VNWKTOKETHGBQD-UHFFFAOYSA-N,reduced,3.895955,7.791967,cc-pvdz,B3LYP


In [5]:
records.to_csv('nwchem-redox-potentials.csv', index=False)